# 업종 확대 테스트 노트북

**목적:** 다양한 업종에 대한 이미지 생성용 프롬프트 생성 시스템 테스트

**테스트 대상:**
- `src/generation/image_generation/prompt/` 모듈들
- 현재 industries.yaml (10개 업종)
- 신규 industries.yaml (247개 업종) 구조 호환성
- import 파일 및 클래스 목록
```python
    # Python
    from src.generation.image_generation.prompt.config_loader import (
        IndustryConfigLoader,
        PromptGenerator
    )
    from src.generation.image_generation.prompt.prompt_templates import (
        HybridPromptBuilder,
        PromptStructure
    )
    from src.generation.image_generation.prompt.style_router import StyleRouter
```

---

## 1. 환경 설정

In [4]:
# 프로젝트 루트 경로 설정
import sys
from pathlib import Path

# 프로젝트 루트 찾기 (codeit_ad_smallbiz)
current_path = Path.cwd()
project_root = current_path

# scripts/신승목에서 실행 시 상위로 이동
while project_root.name != "codeit_ad_smallbiz" and project_root.parent != project_root:
    project_root = project_root.parent

# 프로젝트 루트가 없으면 현재 경로 기준으로 설정
if project_root.name != "codeit_ad_smallbiz":
    # 노트북 경로에서 상대 경로로 프로젝트 루트 찾기
    project_root = Path.cwd().parent.parent

# sys.path에 프로젝트 루트 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"프로젝트 루트: {project_root}")
print(f"sys.path: {sys.path[:3]}...")

프로젝트 루트: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz
sys.path: ['c:\\Users\\USER\\_codeit\\_final_project\\codeit_ad_smallbiz', 'C:\\Users\\USER\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\USER\\AppData\\Local\\Programs\\Python\\Python312\\DLLs']...


In [5]:
# 필요한 라이브러리 import
import json
import yaml
from pprint import pprint

# 프롬프트 모듈 import
try:
    from src.generation.image_generation.prompt.config_loader import (
        IndustryConfigLoader,
        PromptGenerator
    )
    from src.generation.image_generation.prompt.prompt_templates import (
        HybridPromptBuilder,
        PromptStructure
    )
    from src.generation.image_generation.prompt.style_router import StyleRouter
    print("✅ 모든 모듈 import 성공!")
except ImportError as e:
    print(f"❌ Import 오류: {e}")
    print("\n프로젝트 루트 경로를 확인하세요.")

✅ 모든 모듈 import 성공!


## 2. 신규 industries.yaml 구조 분석

In [8]:
# 신규 industries.yaml 로드 (이후 버전)
new_yaml_path = project_root / "scripts" / "신승목" / "industries.yaml"

with open(new_yaml_path, 'r', encoding='utf-8') as f:
    new_config = yaml.safe_load(f)

print(f"신규 버전: {new_config.get('metadata', {}).get('version', 'N/A')}")
print(f"총 등급 수: {new_config.get('metadata', {}).get('total_grades', 'N/A')}")
print(f"총 하위 그룹 수: {new_config.get('metadata', {}).get('total_subgroups', 'N/A')}")
print(f"총 업종 수: {new_config.get('metadata', {}).get('total_businesses', 'N/A')}")

신규 버전: 3.0.0
총 등급 수: 6
총 하위 그룹 수: 18
총 업종 수: 247


In [9]:
# 등급별 구조 확인
grades = ['s_grade', 'a_grade', 'b_grade', 'c_grade', 'd_grade', 'e_grade']

print("=== 등급별 구조 ===")
for grade_key in grades:
    grade_data = new_config.get(grade_key, {})
    if not grade_data:
        continue
    
    print(f"\n{grade_key.upper()}:")
    print(f"  이름: {grade_data.get('name', 'N/A')}")
    print(f"  한글명: {grade_data.get('name_ko', 'N/A')}")
    print(f"  총 업종: {grade_data.get('total_businesses', 'N/A')}개")
    
    # 하위 그룹 목록
    subgroups = [k for k, v in grade_data.items() 
                 if isinstance(v, dict) and 'prompt_template' in v]
    print(f"  하위 그룹: {subgroups}")

=== 등급별 구조 ===

S_GRADE:
  이름: S Grade - Sensory & Taste Stimulation
  한글명: S등급 - 감성 및 미각 자극
  총 업종: 70개
  하위 그룹: ['s1_hot_cooking', 's2_freshness', 's3_emotional', 's4_neat_variety']

A_GRADE:
  이름: A Grade - Visual Transformation & Self-Care
  한글명: A등급 - 비주얼 변화 및 자기관리
  총 업종: 50개
  하위 그룹: ['a1_beauty', 'a2_wellness', 'a3_fashion', 'a4_delicate_care']

B_GRADE:
  이름: B Grade - Space Experience & Education
  한글명: B등급 - 공간 체험 및 교육
  총 업종: 45개
  하위 그룹: ['b1_learning', 'b2_leisure', 'b3_sports']

C_GRADE:
  이름: C Grade - Trust-Based Professional Services
  한글명: C등급 - 신뢰 기반 전문 서비스
  총 업종: 50개
  하위 그룹: ['c1_office', 'c2_medical', 'c3_technical']

D_GRADE:
  이름: D Grade - Purpose-Driven Retail & Equipment
  한글명: D등급 - 목적형 소매 및 장비
  총 업종: 30개
  하위 그룹: ['d1_equipment', 'd2_appliances']

E_GRADE:
  이름: E Grade - Infrastructure & Special Purpose
  한글명: E등급 - 인프라 및 특수 목적
  총 업종: 2개
  하위 그룹: ['e1_infrastructure', 'e2_ceremony']


In [10]:
# 특정 하위 그룹 상세 확인
sample_grade = "s_grade"
sample_subgroup = "s1_hot_cooking"

subgroup_data = new_config.get(sample_grade, {}).get(sample_subgroup, {})

print(f"=== {sample_subgroup} 하위 그룹 상세 ===")
print(f"\n이름: {subgroup_data.get('name', 'N/A')}")
print(f"한글명: {subgroup_data.get('name_ko', 'N/A')}")
print(f"업종 수: {subgroup_data.get('count', 'N/A')}")
print(f"Visual Core: {subgroup_data.get('visual_core', 'N/A')}")

# 포함된 업종들
businesses = subgroup_data.get('businesses', [])
print(f"\n포함 업종 ({len(businesses)}개):")
for i, biz in enumerate(businesses[:10], 1):  # 처음 10개만
    print(f"  {i}. {biz}")
if len(businesses) > 10:
    print(f"  ... 외 {len(businesses) - 10}개")

=== s1_hot_cooking 하위 그룹 상세 ===

이름: S-1: Hot Cooking & Juicy Emphasis
한글명: S-1: 고온 조리 및 육즙 강조
업종 수: 19
Visual Core: Juiciness, steam rising, charcoal grill marks, warm lighting

포함 업종 (19개):
  1. Pork BBQ/Braised
  2. Beef BBQ/Braised
  3. Gopchang Hotpot/Grilled
  4. Chicken/Duck Grilled
  5. Fried Chicken
  6. Pizza
  7. Burger
  8. Malatang/Hotpot
  9. Pasta/Steak
  10. Western-style Restaurant
  ... 외 9개


## 3. IndustryConfigLoader 테스트 (수정 버전)

In [11]:
# IndustryConfigLoader 초기화
loader = IndustryConfigLoader()

print("=== IndustryConfigLoader 테스트 ===")
print(f"\n사용 가능한 업종: {loader.get_industry_names()}")

industries.yaml 파일 주소: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\src\generation\image_generation\prompt\config\industries.yaml
=== IndustryConfigLoader 테스트 ===

사용 가능한 업종: ['s1_hot_cooking', 's2_freshness', 's3_emotional', 's4_neat_variety', 'a1_beauty', 'a2_wellness', 'a3_fashion', 'a4_delicate_care', 'b1_learning', 'b2_leisure', 'b3_sports', 'c1_office', 'c2_medical', 'c3_technical', 'd1_equipment', 'd2_appliances', 'e1_infrastructure', 'e2_ceremony']


In [12]:
# 업종 감지 테스트
test_inputs = [
    "카페 신메뉴 딸기라떼 홍보",
    "헬스장 다이어트 프로모션",
    "빵집 갓 구운 크루아상",
    "미용실 봄 헤어스타일 이벤트",
    "네일샵 봄맞이 특가",
    "꽃집 발렌타인 데이 장미 부케",
    "세탁소 양복 드라이클리닝",
    "옷가게 신상 봄 컬렉션",
    "레스토랑 스테이크 디너 코스",
    "건설 현장 안전 장비",  # general로 감지될 것
]

print("=== 업종 감지 테스트 ===")
for test_input in test_inputs:
    detected = loader.detect_industry(test_input)
    print(f"\n입력: {test_input}")
    print(f"감지: {detected}")

=== 업종 감지 테스트 ===

입력: 카페 신메뉴 딸기라떼 홍보
감지: s3_emotional

입력: 헬스장 다이어트 프로모션
감지: a2_wellness

입력: 빵집 갓 구운 크루아상
감지: s3_emotional

입력: 미용실 봄 헤어스타일 이벤트
감지: a1_beauty

입력: 네일샵 봄맞이 특가
감지: a1_beauty

입력: 꽃집 발렌타인 데이 장미 부케
감지: a4_delicate_care

입력: 세탁소 양복 드라이클리닝
감지: a4_delicate_care

입력: 옷가게 신상 봄 컬렉션
감지: a3_fashion

입력: 레스토랑 스테이크 디너 코스
감지: s1_hot_cooking

입력: 건설 현장 안전 장비
감지: s4_neat_variety


## 4. PromptGenerator 테스트

In [13]:
# PromptGenerator 초기화
generator = PromptGenerator()

print("=== PromptGenerator 테스트 ===")

industries.yaml 파일 주소: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz\src\generation\image_generation\prompt\config\industries.yaml
=== PromptGenerator 테스트 ===


In [14]:
# 각 업종별 프롬프트 생성 테스트
test_cases = {
    "cafe": {
        "product": "strawberry latte",
        "surface": "marble table",
        "theme": "warm",
        "mood": "cozy",
        "style": "realistic"
    },
    "gym": {
        "person_type": "athletic man",
        "activity": "barbell squat",
        "focus": "muscle definition",
        "style": "realistic"
    },
    "bakery": {
        "product": "fresh croissant",
        "presentation": "on wooden board",
        "theme": "rustic",
        "time": "morning",
        "style": "realistic"
    },
    "restaurant": {
        "dish": "pasta carbonara",
        "plating": "elegantly plated",
        "cuisine_style": "italian",
        "style": "realistic"
    },
    "hair_salon": {
        "service": "hair styling",
        "result": "stylish haircut",
        "style": "realistic"
    },
    "nail_salon": {
        "service": "gel manicure",
        "design": "french tip",
        "style": "realistic"
    },
    "flower_shop": {
        "flowers": "red roses bouquet",
        "arrangement": "wrapped elegantly",
        "style": "realistic"
    },
    "laundry": {
        "item": "white shirt",
        "state": "freshly pressed",
        "quality": "crisp and clean",
        "style": "realistic"
    },
    "clothing_store": {
        "item": "spring dress",
        "display_method": "on mannequin",
        "style": "realistic"
    },
    "general": {
        "subject": "business service",
        "setting": "professional office",
        "style": "realistic"
    }
}

print("=== 업종별 프롬프트 생성 결과 ===")

for industry, user_input in test_cases.items():
    try:
        result = generator.generate(
            industry=industry,
            user_input=user_input
        )
        
        print(f"\n{'='*60}")
        print(f"[{industry.upper()}]")
        print(f"입력: {user_input}")
        print(f"\n✅ Positive Prompt:")
        print(f"   {result['positive'][:150]}..." if len(result['positive']) > 150 else f"   {result['positive']}")
        print(f"\n❌ Negative Prompt:")
        print(f"   {result['negative']}")
        print(f"\nStyle: {result['style']}")
        
    except Exception as e:
        print(f"\n{'='*60}")
        print(f"[{industry.upper()}] ❌ 오류: {e}")

=== 업종별 프롬프트 생성 결과 ===

[CAFE]
입력: {'product': 'strawberry latte', 'surface': 'marble table', 'theme': 'warm', 'mood': 'cozy', 'style': 'realistic'}

✅ Positive Prompt:
   Professional commercial photography of strawberry latte. placed on marble table. placed on marble table. soft natural window light streaming gently. c...

❌ Negative Prompt:
   

Style: realistic

[GYM]
입력: {'person_type': 'athletic man', 'activity': 'barbell squat', 'focus': 'muscle definition', 'style': 'realistic'}

✅ Positive Prompt:
   Professional commercial photography of athletic man. shown barbell squat. dramatic side lighting highlighting muscle definition. shot on Canon EOS R5 ...

❌ Negative Prompt:
   

Style: realistic

[BAKERY]
입력: {'product': 'fresh croissant', 'presentation': 'on wooden board', 'theme': 'rustic', 'time': 'morning', 'style': 'realistic'}

✅ Positive Prompt:
   Professional commercial photography of fresh croissant. shown on wooden board. soft natural window light streaming gently. rus

## 5. 스타일별 프롬프트 생성 테스트

In [15]:
# 동일 업종, 다른 스타일 테스트

# 현재 StyleRouter 클래스에 get_style_technical 함수 제거

styles = ["realistic", "semi_realistic", "anime"]

base_input = {
    "product": "strawberry latte",
    "surface": "marble table"
}

print("=== 스타일별 프롬프트 비교 (cafe) ===")

for style in styles:
    user_input = {**base_input, "style": style}
    result = generator.generate(
        industry="cafe",
        user_input=user_input
    )
    
    print(f"\n{'='*60}")
    print(f"스타일: {style}")
    print(f"\nPositive: {result['positive'][:100]}...")
    print(f"Negative: {result['negative'][:80]}...")

=== 스타일별 프롬프트 비교 (cafe) ===

스타일: realistic

Positive: Professional commercial photography of strawberry latte. placed on marble table. placed on marble ta...
Negative: ...

스타일: semi_realistic

Positive: Highly detailed digital artwork of strawberry latte. placed on marble table. placed on marble table....
Negative: ...

스타일: anime

Positive: Vibrant anime style illustration of strawberry latte. placed on marble table. placed on marble table...
Negative: ...


## 6. StyleRouter 테스트

In [16]:
# StyleRouter 기능 테스트
print("=== StyleRouter 테스트 ===")

print(f"\n지원 스타일: {StyleRouter.get_supported_styles()}")

# 스타일별 prefix 확인
print("\n스타일별 Prefix:")
for style in StyleRouter.get_supported_styles():
    prefix = StyleRouter.get_style_prefix(style)
    print(f"  {style}: {prefix}")

=== StyleRouter 테스트 ===

지원 스타일: ['realistic', 'ultra_realistic', 'semi_realistic', 'anime']

스타일별 Prefix:
  realistic: Professional commercial photography of
  ultra_realistic: Professional commercial photography of
  semi_realistic: Highly detailed digital artwork of
  anime: Vibrant anime style illustration of


In [17]:
# Subject Phrase 빌드 테스트
print("=== Subject Phrase 빌드 테스트 ===")

for style in ["realistic", "anime"]:
    phrase = StyleRouter.build_subject_phrase(
        style=style,
        subject="strawberry latte",
        additional_context="on marble table in cozy cafe"
    )
    print(f"\n{style}:")
    print(f"  {phrase}")

=== Subject Phrase 빌드 테스트 ===

realistic:
  Professional commercial photography of strawberry latte. on marble table in cozy cafe

anime:
  Vibrant anime style illustration of strawberry latte. on marble table in cozy cafe


## 7. 신규 YAML 구조 호환성 테스트

신규 industries.yaml의 하위 그룹에서 `prompt_template`을 추출하여
기존 `HybridPromptBuilder`와 호환되는지 테스트합니다.

In [18]:
# 신규 YAML에서 하위 그룹 템플릿 추출 테스트
def get_subgroup_template(config: dict, grade_key: str, subgroup_key: str) -> dict:
    """신규 YAML에서 하위 그룹 템플릿 추출"""
    grade_data = config.get(grade_key, {})
    subgroup_data = grade_data.get(subgroup_key, {})
    return subgroup_data

# S1 (고온 조리) 템플릿 추출
s1_template = get_subgroup_template(new_config, "s_grade", "s1_hot_cooking")

print("=== 신규 YAML 템플릿 호환성 테스트 ===")
print(f"\n하위 그룹: s1_hot_cooking")
print(f"템플릿 키: {list(s1_template.get('prompt_template', {}).keys())}")

=== 신규 YAML 템플릿 호환성 테스트 ===

하위 그룹: s1_hot_cooking
템플릿 키: ['subject_patterns', 'setting_patterns', 'composition_keywords', 'lighting_phrases', 'style_keywords', 'color_phrases', 'details_keywords', 'technical_keywords']


In [19]:
# HybridPromptBuilder로 신규 템플릿 빌드 테스트
try:
    builder = HybridPromptBuilder(s1_template)
    
    user_input = {
        "dish": "grilled pork belly",
        "presentation": "sizzling on hot plate",
        "theme": "warm",
        "mood": "appetizing"
    }
    
    structure = builder.build_from_user_input(user_input)
    prompt = structure.build()
    
    print("✅ HybridPromptBuilder 호환성 확인!")
    print(f"\n생성된 프롬프트:")
    print(f"  {prompt}")
    
except Exception as e:
    print(f"❌ 호환성 오류: {e}")

✅ HybridPromptBuilder 호환성 확인!

생성된 프롬프트:
  Professional commercial photography of grilled pork belly. shown sizzling on hot plate. warm cinematic lighting from above. appetizing and warm atmosphere. shot on Canon EOS R5 with 85mm lens, natural lighting, shallow depth of field. with glistening oil on surface, steam rising elegantly, charcoal grill marks visible.


In [20]:
# 다양한 하위 그룹 테스트
test_subgroups = [
    ("s_grade", "s1_hot_cooking", {"dish": "fried chicken", "presentation": "crispy golden"}),
    ("s_grade", "s2_freshness", {"dish": "fresh sashimi", "presentation": "on ice"}),
    ("s_grade", "s3_emotional", {"product": "cappuccino", "theme": "cozy"}),
    ("a_grade", "a1_beauty", {"service": "hair styling", "result": "elegant updo"}),
    ("a_grade", "a2_wellness", {"activity": "yoga pose", "focus": "flexibility"}),
    ("b_grade", "b1_learning", {"setting": "bright study room", "atmosphere": "focused"}),
    ("c_grade", "c1_office", {"service": "consultation", "setting": "modern office"}),
]

print("=== 다양한 하위 그룹 프롬프트 생성 테스트 ===")

for grade_key, subgroup_key, user_input in test_subgroups:
    template = get_subgroup_template(new_config, grade_key, subgroup_key)
    
    if not template.get('prompt_template'):
        print(f"\n[{subgroup_key}] ⚠️ prompt_template 없음")
        continue
    
    try:
        builder = HybridPromptBuilder(template)
        structure = builder.build_from_user_input(user_input)
        prompt = structure.build()
        
        print(f"\n{'='*60}")
        print(f"[{subgroup_key}] ({template.get('name_ko', 'N/A')})")
        print(f"입력: {user_input}")
        print(f"프롬프트: {prompt[:120]}..." if len(prompt) > 120 else f"프롬프트: {prompt}")
        
    except Exception as e:
        print(f"\n[{subgroup_key}] ❌ 오류: {e}")

=== 다양한 하위 그룹 프롬프트 생성 테스트 ===

[s1_hot_cooking] (S-1: 고온 조리 및 육즙 강조)
입력: {'dish': 'fried chicken', 'presentation': 'crispy golden'}
프롬프트: Professional commercial photography of fried chicken. shown crispy golden. warm cinematic lighting from above. shot on C...

[s2_freshness] (S-2: 신선도 및 색감 강조)
입력: {'dish': 'fresh sashimi', 'presentation': 'on ice'}
프롬프트: Professional commercial photography of fresh sashimi. shown on ice. bright studio lighting emphasizing freshness. shot o...

[s3_emotional] (S-3: 감성 및 부드러운 질감)
입력: {'product': 'cappuccino', 'theme': 'cozy'}
프롬프트: Professional commercial photography of cappuccino. soft natural window light streaming gently. cozy mood. shot on Canon ...

[a1_beauty] (A-1: 인물 중심 뷰티 및 에스테틱)
입력: {'service': 'hair styling', 'result': 'elegant updo'}
프롬프트: Professional commercial photography of subject. soft rim lighting highlighting features. shot on Canon EOS R5 with 85mm ...

[a2_wellness] (A-2: 신체 변화 및 웰니스)
입력: {'activity': 'yoga pose', 'focus': 'flexib

## 8. 한글 입력 → 업종 감지 → 프롬프트 생성 통합 테스트

실제 사용 시나리오를 시뮬레이션합니다.

In [21]:
# 실제 사용 시나리오 테스트
real_world_inputs = [
    # S등급: 감성 및 미각 자극
    "삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습",
    "횟집 신선한 회 모듬 광고, 얼음 위에 회가 올라간 사진",
    "카페 봄 시즌 딸기 음료 출시 홍보",
    "한식 정식 백반집 광고, 반찬이 푸짐하게 차려진 상차림",
    
    # A등급: 비주얼 변화 및 자기관리
    "미용실 봄 시즌 헤어컬러 이벤트, 예쁜 염색 모습",
    "헬스장 PT 프로그램 홍보, 운동하는 모습",
    "여성복 봄 신상 컬렉션, 화사한 원피스",
    "꽃집 어버이날 카네이션 부케 광고",
    
    # B등급: 공간 체험 및 교육
    "독서실 조용하고 쾌적한 스터디 공간 홍보",
    "펜션 가족 여행 숙소 광고, 넓고 아늑한 객실",
    "골프 연습장 레슨 프로그램 홍보",
    
    # C등급: 신뢰 기반 전문 서비스
    "변호사 사무실 법률 상담 서비스 광고",
    "치과 임플란트 시술 안내",
    "자동차 정비소 엔진 오일 교환 프로모션",
    
    # D등급: 목적형 소매
    "타이어 할인 판매 광고",
    "컴퓨터 수리 서비스 홍보",
    
    # E등급: 인프라
    "주유소 위치 안내",
]

print("=== 실제 사용 시나리오 테스트 ===")

for korean_input in real_world_inputs:
    # 1. 업종 감지
    detected_industry = loader.detect_industry(korean_input)
    
    # 2. 간단한 키워드 추출 (실제로는 GPT가 수행)
    # 여기서는 테스트를 위해 간단히 처리
    simple_input = {
        "subject": korean_input[:20],
        "style": "realistic"
    }
    
    # 3. 프롬프트 생성 (detected_industry 사용)
    try:
        result = generator.generate(
            industry=detected_industry,
            user_input=simple_input
        )
        
        print(f"\n{'='*60}")
        print(f"입력: {korean_input}")
        print(f"감지 업종: {detected_industry}")
        print(f"프롬프트: {result['positive'][:80]}...")
        
    except Exception as e:
        print(f"\n입력: {korean_input}")
        print(f"❌ 오류: {e}")

=== 실제 사용 시나리오 테스트 ===

입력: 삼겹살 맛집 홍보, 불판에서 지글지글 굽는 모습
감지 업종: s1_hot_cooking
프롬프트: Professional commercial photography of 삼겹살 맛집 홍보, 불판에서 지글지글. warm cinematic ligh...

입력: 횟집 신선한 회 모듬 광고, 얼음 위에 회가 올라간 사진
감지 업종: s2_freshness
프롬프트: Professional commercial photography of 횟집 신선한 회 모듬 광고, 얼음 위. bright studio light...

입력: 카페 봄 시즌 딸기 음료 출시 홍보
감지 업종: s3_emotional
프롬프트: Professional commercial photography of 카페 봄 시즌 딸기 음료 출시 홍보. soft natural window ...

입력: 한식 정식 백반집 광고, 반찬이 푸짐하게 차려진 상차림
감지 업종: s4_neat_variety
프롬프트: Professional commercial photography of 한식 정식 백반집 광고, 반찬이 푸짐. home-style warm ove...

입력: 미용실 봄 시즌 헤어컬러 이벤트, 예쁜 염색 모습
감지 업종: a1_beauty
프롬프트: Professional commercial photography of 미용실 봄 시즌 헤어컬러 이벤트, 예. soft rim lighting h...

입력: 헬스장 PT 프로그램 홍보, 운동하는 모습
감지 업종: a2_wellness
프롬프트: Professional commercial photography of 헬스장 PT 프로그램 홍보, 운동하는. dramatic side light...

입력: 여성복 봄 신상 컬렉션, 화사한 원피스
감지 업종: a3_fashion
프롬프트: Professional commercial photography of 여성복 봄 신상 컬렉션, 화사한 원피. natural sunl

## 9. 신규 YAML용 업종 감지 함수 (korean_keywords 필드 활용)

신규 `industries.yaml`에 추가된 `korean_keywords` 필드를 활용하여 247개 업종을 지원하는 새로운 업종 감지 로직을 테스트합니다.

In [22]:
# 신규 업종 감지 함수 - YAML의 korean_keywords 필드 활용
def build_subgroup_cache(config: dict) -> dict:
    """하위 그룹 캐시 생성"""
    grades = ['s_grade', 'a_grade', 'b_grade', 'c_grade', 'd_grade', 'e_grade']
    cache = {}
    
    for grade_key in grades:
        grade_data = config.get(grade_key, {})
        for key, value in grade_data.items():
            if isinstance(value, dict) and 'prompt_template' in value:
                cache[key] = (grade_key, value)
    
    return cache

def build_keyword_map_from_yaml(config: dict, subgroup_cache: dict) -> dict:
    """
    YAML의 korean_keywords 필드를 직접 활용하여 키워드 맵 구축
    
    이제 하드코딩된 한글 키워드 대신 YAML 파일에서 직접 읽어옴
    """
    keyword_map = {}
    
    for subgroup_code, (grade_key, subgroup_data) in subgroup_cache.items():
        keywords = []
        
        # 1. 영어 keywords 필드
        english_keywords = subgroup_data.get('keywords', [])
        keywords.extend([kw.lower() for kw in english_keywords])
        
        # 2. korean_keywords 필드 (YAML에서 직접 로드)
        korean_keywords = subgroup_data.get('korean_keywords', [])
        keywords.extend(korean_keywords)
        
        # 3. businesses 필드에서 추가 키워드 추출
        businesses = subgroup_data.get('businesses', [])
        for business in businesses:
            words = business.lower().replace('/', ' ').split()
            keywords.extend(words)
        
        keyword_map[subgroup_code] = list(set(keywords))
    
    return keyword_map

def detect_industry_v2(user_input: str, keyword_map: dict) -> str:
    """
    신규 업종 감지 함수 (YAML korean_keywords 활용)
    
    Args:
        user_input: 사용자 입력 텍스트
        keyword_map: build_keyword_map_from_yaml()로 생성된 키워드 맵
        
    Returns:
        str: 감지된 하위 그룹 코드 또는 기본값 "s4_neat_variety"
    """
    user_input_lower = user_input.lower()
    
    scores = {}
    for subgroup_code, keywords in keyword_map.items():
        score = sum(1 for kw in keywords if kw in user_input_lower)
        if score > 0:
            scores[subgroup_code] = score
    
    if scores:
        return max(scores.items(), key=lambda x: x[1])[0]
    
    return "s4_neat_variety"  # 기본값

# 캐시 및 키워드 맵 생성
subgroup_cache = build_subgroup_cache(new_config)
keyword_map = build_keyword_map_from_yaml(new_config, subgroup_cache)

print(f"총 하위 그룹 수: {len(subgroup_cache)}")
print(f"하위 그룹 목록: {list(subgroup_cache.keys())}")

# korean_keywords 로드 확인
print(f"\n=== korean_keywords 로드 확인 ===")
for subgroup_code in list(subgroup_cache.keys())[:3]:
    _, subgroup_data = subgroup_cache[subgroup_code]
    korean_kw = subgroup_data.get('korean_keywords', [])
    print(f"{subgroup_code}: {korean_kw[:5]}..." if len(korean_kw) > 5 else f"{subgroup_code}: {korean_kw}")

총 하위 그룹 수: 18
하위 그룹 목록: ['s1_hot_cooking', 's2_freshness', 's3_emotional', 's4_neat_variety', 'a1_beauty', 'a2_wellness', 'a3_fashion', 'a4_delicate_care', 'b1_learning', 'b2_leisure', 'b3_sports', 'c1_office', 'c2_medical', 'c3_technical', 'd1_equipment', 'd2_appliances', 'e1_infrastructure', 'e2_ceremony']

=== korean_keywords 로드 확인 ===
s1_hot_cooking: ['삼겹살', '고기', '구이', '소고기', '돼지고기']...
s2_freshness: ['회', '횟집', '초밥', '사시미', '해산물']...
s3_emotional: ['카페', '커피', '라떼', '아메리카노', '에스프레소']...


In [23]:
# 신규 업종 감지 테스트 (YAML korean_keywords 활용)
test_inputs_v2 = [
    "삼겹살 맛집 홍보",
    "횟집 신선한 회",
    "카페 딸기라떼",
    "한식 백반 정식",
    "미용실 헤어 염색",
    "헬스장 PT 프로그램",
    "여성복 봄 신상",
    "꽃집 장미 부케",
    "독서실 스터디",
    "호텔 숙박",
    "골프 연습장",
    "변호사 법률 상담",
    "치과 임플란트",
    "자동차 정비",
    "타이어 교체",
    "컴퓨터 수리",
    "주유소",
    "장례식장",
]

print("=== 신규 업종 감지 테스트 (YAML korean_keywords 활용) ===")
for test_input in test_inputs_v2:
    detected = detect_industry_v2(test_input, keyword_map)
    grade_key, subgroup_data = subgroup_cache.get(detected, (None, {}))
    name_ko = subgroup_data.get('name_ko', 'N/A')
    
    print(f"\n입력: {test_input}")
    print(f"감지: {detected} ({name_ko})")

=== 신규 업종 감지 테스트 (YAML korean_keywords 활용) ===

입력: 삼겹살 맛집 홍보
감지: s1_hot_cooking (S-1: 고온 조리 및 육즙 강조)

입력: 횟집 신선한 회
감지: s2_freshness (S-2: 신선도 및 색감 강조)

입력: 카페 딸기라떼
감지: s3_emotional (S-3: 감성 및 부드러운 질감)

입력: 한식 백반 정식
감지: s4_neat_variety (S-4: 정갈함과 다채로움)

입력: 미용실 헤어 염색
감지: a1_beauty (A-1: 인물 중심 뷰티 및 에스테틱)

입력: 헬스장 PT 프로그램
감지: a2_wellness (A-2: 신체 변화 및 웰니스)

입력: 여성복 봄 신상
감지: a3_fashion (A-3: 패션 및 스타일 아이템)

입력: 꽃집 장미 부케
감지: a4_delicate_care (A-4: 섬세한 케어 및 감성 소품)

입력: 독서실 스터디
감지: b1_learning (B-1: 학습/몰입)

입력: 호텔 숙박
감지: b2_leisure (B-2: 여가/스테이)

입력: 골프 연습장
감지: b3_sports (B-3: 스포츠/활동)

입력: 변호사 법률 상담
감지: c1_office (C-1: 전문 사무)

입력: 치과 임플란트
감지: c2_medical (C-2: 의료/케어)

입력: 자동차 정비
감지: s4_neat_variety (S-4: 정갈함과 다채로움)

입력: 타이어 교체
감지: d1_equipment (D-1: 중장비/부품)

입력: 컴퓨터 수리
감지: d2_appliances (D-2: 일반 기기)

입력: 주유소
감지: e1_infrastructure (E-1: 편의/인프라)

입력: 장례식장
감지: e2_ceremony (E-2: 예절/의례)


## 10. 성능 및 결과 요약

In [27]:
# 테스트 결과 요약
print("="*60)
print("테스트 결과 요약")
print("="*60)

print("\n[신규 버전 (v3.0.0)]")
print(f"  - 지원 등급: 6개 (S, A, B, C, D, E)")
print(f"  - 하위 그룹: {len(subgroup_cache)}개")
print(f"  - 총 업종: 247개")

# korean_keywords 필드 확인
korean_kw_count = sum(
    1 for _, (_, data) in subgroup_cache.items() 
    if data.get('korean_keywords')
)
print(f"  - korean_keywords 필드 보유 그룹: {korean_kw_count}/{len(subgroup_cache)}개")

print("\n[호환성 확인]")
print("  ✅ HybridPromptBuilder: 신규 YAML 템플릿과 호환")
print("  ✅ PromptStructure: 변경 없이 사용 가능")
print("  ✅ StyleRouter: 변경 없이 사용 가능")
print("  ✅ korean_keywords: YAML에서 직접 로드 가능")
print("  ⚠️  IndustryConfigLoader: 계층 구조 지원 필요 (로컬 코드 수정 완료, 확인 후 적용 필요)")

테스트 결과 요약

[신규 버전 (v3.0.0)]
  - 지원 등급: 6개 (S, A, B, C, D, E)
  - 하위 그룹: 18개
  - 총 업종: 247개
  - korean_keywords 필드 보유 그룹: 18/18개

[호환성 확인]
  ✅ HybridPromptBuilder: 신규 YAML 템플릿과 호환
  ✅ PromptStructure: 변경 없이 사용 가능
  ✅ StyleRouter: 변경 없이 사용 가능
  ✅ korean_keywords: YAML에서 직접 로드 가능
  ⚠️  IndustryConfigLoader: 계층 구조 지원 필요 (로컬 코드 수정 완료, 확인 후 적용 필요)


In [25]:
print("\n" + "="*60)
print("테스트 완료!")
print("="*60)


테스트 완료!
